In [2]:
import argparse
import csv
import numpy as np
import os
import pandas as pd
from pprint import pprint
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

gpt35_df = pd.read_csv('../conditional/data/112_gsm8k_gpt35_cot_onesent_responses.csv')

model_name = "mistralai/Mistral-7B-Instruct-v0.2"
# model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

Loading checkpoint shards: 100%|██████████| 3/3 [00:25<00:00,  8.55s/it]


In [8]:
questions = gpt35_df['Question'].tolist() 


for i, answer in enumerate(gpt35_df['Answer'].tolist()):
    print(i)
    pprint(questions[i])
    print()
    pprint(answer)
    for _ in range(3):
        print()

0
('Natalia sold clips to 48 of her friends in April, and then she sold half as '
 'many clips in May. How many clips did Natalia sell altogether in April and '
 'May?')

('Natalia sold 48/2 = <<48/2=24>>24 clips in May.\n'
 'Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n'
 '#### 72')



1
('Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes '
 'of babysitting. How much did she earn?')

('Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.\n'
 'Working 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.\n'
 '#### 10')



2
('Betty is saving money for a new wallet which costs $100. Betty has only half '
 'of the money she needs. Her parents decided to give her $15 for that '
 'purpose, and her grandparents twice as much as her parents. How much more '
 'money does Betty need to buy the wallet?')

('In the beginning, Betty has only 100 / 2 = $<<100/2=50>>50.\n'
 "Betty's grandparents gave her 15 * 2 = $<<15*2=30>>30.\n"
 'This means, Betty n

In [11]:
overtime_question = questions[9]
overtime_question

'Tina makes $18.00 an hour.  If she works more than 8 hours per shift, she is eligible for overtime, which is paid by your hourly wage + 1/2 your hourly wage.  If she works 10 hours every day for 5 days, how much money does she make?'

In [12]:
num_samples = 2

In [19]:
from utils import get_prompt_message

question_vector = tokenizer.apply_chat_template(get_prompt_message(overtime_question, 0), add_generation_prompt=True, return_tensors="pt")
input_tensor = question_vector.repeat(num_samples, 1)

PROMPT:  Q: Tina makes $18.00 an hour.  If she works more than 8 hours per shift, she is eligible for overtime, which is paid by your hourly wage + 1/2 your hourly wage.  If she works 10 hours every day for 5 days, how much money does she make?
A: Let's think step by step.


In [20]:
outputs = model.generate(
                input_tensor.to(model.device), 
                max_new_tokens=1000, 
                return_dict_in_generate=True, 
                output_scores=True,
                do_sample=True,
                temperature=0.7,
                top_k=40,
            )

In [22]:
outputs.sequences.shape

torch.Size([2, 509])

In [30]:
for i, sol in enumerate(tokenizer.batch_decode(outputs.sequences)):
    print("SOLUTION ", i)
    pprint(sol)
    for _ in range(5):
        print()

SOLUTION  0
('<s> [INST] Q: Tina makes $18.00 an hour.  If she works more than 8 hours per '
 'shift, she is eligible for overtime, which is paid by your hourly wage + 1/2 '
 'your hourly wage.  If she works 10 hours every day for 5 days, how much '
 'money does she make?\n'
 "A: Let's think step by step. [/INST] First, let's calculate how much Tina "
 'earns for working a regular 8-hour shift:\n'
 '\n'
 'Regular hours earned = Hourly wage * Hours worked per day * Number of days\n'
 'Regular hours earned = $18.00 * 8 * 5\n'
 'Regular hours earned = $720.00\n'
 '\n'
 "Next, let's calculate how many overtime hours she works in total:\n"
 '\n'
 'Overtime hours = Total hours worked - Regular hours\n'
 'Overtime hours = 10 hours * 5 days - 8 hours * 5 days\n'
 'Overtime hours = 10 * 5 - 8 * 5\n'
 'Overtime hours = 50 hours - 40 hours\n'
 'Overtime hours = 10 hours\n'
 '\n'
 "Now, let's calculate the overtime pay for those 10 hours:\n"
 '\n'
 'Overtime pay per hour = Hourly wage + 1/2 hourly

In [33]:
for i, sol in enumerate(tokenizer.batch_decode(outputs.sequences)):
    print(sol)
    for _ in range(5):
        print()

<s> [INST] Q: Tina makes $18.00 an hour.  If she works more than 8 hours per shift, she is eligible for overtime, which is paid by your hourly wage + 1/2 your hourly wage.  If she works 10 hours every day for 5 days, how much money does she make?
A: Let's think step by step. [/INST] First, let's calculate how much Tina earns for working a regular 8-hour shift:

Regular hours earned = Hourly wage * Hours worked per day * Number of days
Regular hours earned = $18.00 * 8 * 5
Regular hours earned = $720.00

Next, let's calculate how many overtime hours she works in total:

Overtime hours = Total hours worked - Regular hours
Overtime hours = 10 hours * 5 days - 8 hours * 5 days
Overtime hours = 10 * 5 - 8 * 5
Overtime hours = 50 hours - 40 hours
Overtime hours = 10 hours

Now, let's calculate the overtime pay for those 10 hours:

Overtime pay per hour = Hourly wage + 1/2 hourly wage
Overtime pay per hour = $18.00 + $9.00
Overtime pay per hour = $27.00

Total overtime pay = Overtime pay per 

In [31]:
outputs_2 = model.generate(
                input_tensor.to(model.device), 
                max_new_tokens=1000, 
                return_dict_in_generate=True, 
                output_scores=True,
                do_sample=True,
                temperature=0.7,
                top_k=40,
            )

In [32]:
for i, sol in enumerate(tokenizer.batch_decode(outputs_2.sequences)):
    print("SOLUTION ", i)
    pprint(sol)
    for _ in range(5):
        print()

SOLUTION  0
('<s> [INST] Q: Tina makes $18.00 an hour.  If she works more than 8 hours per '
 'shift, she is eligible for overtime, which is paid by your hourly wage + 1/2 '
 'your hourly wage.  If she works 10 hours every day for 5 days, how much '
 'money does she make?\n'
 "A: Let's think step by step. [/INST] First, let's find out how much Tina "
 'earns for working a regular 8-hour shift:\n'
 '\n'
 'Regular hours pay = Hourly wage * Hours worked per day * Number of working '
 'days\n'
 '                           = $18.00 * 8 * 5\n'
 '                           = $720\n'
 '\n'
 "Next, let's calculate the overtime hours and the corresponding pay:\n"
 '\n'
 'Overtime hours = Total hours worked - Regular hours\n'
 '                          = 10 hours/day * 5 days/week - 8 hours/day * 5 '
 'days/week\n'
 '                          = 2 hours/day * 5 days/week\n'
 '                          = 10 hours/week\n'
 '\n'
 'Overtime pay = Overtime hours * (Hourly wage + 1/2 hourly wage)\n'
 '